# Load the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#dataset_path = "/content/drive/MyDrive/Memes dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Define variables to deal with Gemini's request time limit

In [ ]:
from datetime import datetime

global TIME_LAST_REQUEST
global TIME_BETWEEN_REQUESTS

TIME_LAST_REQUEST = datetime.now()
TIME_BETWEEN_REQUESTS = 4.5 #seconds
# Gemini 1.5 Flash -> 15 RPM (1 req. each 4 s)

# Install dependencies

In [ ]:
!pip install frame-semantic-transformer
!pip install cnocr[ort-gpu]

  Using cached pytorch_lightning-1.9.5-py3-none-any.whl.metadata (23 kB)
Using cached pytorch_lightning-1.9.5-py3-none-any.whl (829 kB)
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 2.5.0.post0
    Uninstalling pytorch-lightning-2.5.0.post0:
      Successfully uninstalled pytorch-lightning-2.5.0.post0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cnocr 2.3.1 requires pytorch-lightning>=2.0.0, but you have pytorch-lightning 1.9.5 which is incompatible.
  Using cached pytorch_lightning-2.5.0.post0-py3-none-any.whl.metadata (21 kB)
Using cached pytorch_lightning-2.5.0.post0-py3-none-any.whl (819 kB)
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.9.5
    Uninstalling pytorch-lightning-1.9.5:
      Successfully uninstalled pytorch-lightning-1.9.5
ERROR: pip's dependency 

In [ ]:
import os
import google.generativeai as genai
import re
import requests
import torch
import json
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image, ImageDraw, ImageFilter
from frame_semantic_transformer import FrameSemanticTransformer
from cnocr import CnOcr
import time
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load models

In [ ]:
def load_captioning_model():

  global model, processor, device, torch_dtype

  device = "cuda:0" if torch.cuda.is_available() else "cpu"
  torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

  model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
  processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

load_captioning_model()

def load_llm():

  global large_language_model

  genai.configure(api_key="")
  large_language_model = genai.GenerativeModel("gemini-1.5-flash")

load_llm()

def load_frame_extractor():

  global frame_transformer

  frame_transformer = FrameSemanticTransformer()

load_frame_extractor()

def load_chinese_ocr():

  global ocr

  ocr = CnOcr(det_model_name='ch_PP-OCRv4_det', rec_model_name='ch_PP-OCRv4')

load_chinese_ocr()


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)



# Extract superimposed text

In [ ]:
def ocr_region(image):

    prompt = "<OCR_WITH_REGION>"


    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=4096,
        num_beams=3,
        do_sample=False
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

    parsed_answer = processor.post_process_generation(generated_text, task=prompt, image_size=(image.width, image.height))

    labels = parsed_answer[prompt]['labels']

    text = " ".join(re.sub(r"</?[^>]+>", "", s) for s in labels)

    return text, parsed_answer[prompt]['quad_boxes']

def is_chinese(text):
    """
    Check if the text contains Chinese characters (Simplified or Traditional).
    """
    # Unicode ranges for Chinese characters
    chinese_pattern = re.compile(r'[\u4e00-\u9fff]')
    return bool(chinese_pattern.search(text))

# Define a different text extraction for Chinese memes (Florence doesn't correctly retrieve Chinese text)
def ocr_chinese(image_path):

  output = ocr.ocr(image_path)
  extracted_text = ""
  for out in output:
    sub_text = out['text']
    # Exclude the creator of the meme from the text
    if sub_text[0] != "@" and is_chinese(sub_text):
      extracted_text += sub_text + " "

  return extracted_text

# Text masking

In [ ]:
def mask_image(image, bounds, blur_radius=10):

  blurred_image = image.filter(ImageFilter.GaussianBlur(blur_radius))

  mask = Image.new("L", image.size, 0)
  mask_draw = ImageDraw.Draw(mask)

  for quad in bounds:
    # Draw each quadrilateral on the mask
    polygon = [(quad[i], quad[i + 1]) for i in range(0, len(quad), 2)]
    mask_draw.polygon(polygon, fill=255)

  # Composite the blurred image over the original using the mask
  image.paste(blurred_image, (0, 0), mask)

  return image

# Caption generation

In [ ]:
def generate_caption(image):

    prompt = "<MORE_DETAILED_CAPTION>"

    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=4096,
        num_beams=3,
        do_sample=False
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

    parsed_answer = processor.post_process_generation(generated_text, task=prompt, image_size=(image.width, image.height))

    return parsed_answer[prompt]

# Explaination generation

In [ ]:
def generate_meaning(text, caption):

    global TIME_LAST_REQUEST, TIME_BETWEEN_REQUESTS

    current_time = datetime.now()
    time_from_last_request = current_time - TIME_LAST_REQUEST

    if time_from_last_request.seconds < TIME_BETWEEN_REQUESTS:
        time_to_wait = TIME_BETWEEN_REQUESTS - time_from_last_request.seconds
        time.sleep(time_to_wait)

    TIME_LAST_REQUEST = datetime.now()

    response = large_language_model.generate_content(
    f"Explain the meaning of this meme given the image description: '{caption}' and the superimposed text: '{text}'.",
    generation_config=genai.types.GenerationConfig(
        #candidate_count=1,
        #stop_sequences=["x"],
        #max_output_tokens=100,
        temperature=1.5,
        )
    )

    return response.text

# Full pipeline

In [ ]:
results = []

In [ ]:
def instance_already_processed(results, meme_name, language, instance):
    for result in results:
        if result["Meme name"] == meme_name and result["Language"] == language and result["Instance"] == instance:
            return True
    return False

In [ ]:
root_folder = "/content/drive/MyDrive/Dataset"

subfolders = [item for item in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, item))]
total_memes = len(subfolders)
counter = 0


# Loop through all meme folders
for meme_folder in os.listdir(root_folder):
    meme_path = os.path.join(root_folder, meme_folder)

    if os.path.isdir(meme_path):  # Check if it's a folder
        counter += 1
        print(f"\nProcessing meme: {meme_folder}      {counter}/{total_memes}")
        # Loop through the 'English', 'French', and 'Chinese' subfolders
        for language_folder in ['English', 'French', 'Chinese']:
            language_path = os.path.join(meme_path, language_folder)

            if os.path.isdir(language_path):  # Check if the language folder exists
                print(f"\tProcessing language: {language_folder}")
                # Loop through all images in the language folder
                for instance in os.listdir(language_path):
                    instance_path = os.path.join(language_path, instance)

                    if os.path.isfile(instance_path) and instance.lower().endswith((".png", ".jpg", ".jpeg", ".gif", ".bmp", ".tiff")):  # Check if it's a file (image)
                        # Perform your operation on each image
                        if instance_already_processed(results, meme_folder, language_folder, instance):
                          print(f"\t\tInstance {instance} already processed.")
                          continue
                        else:
                          print(f"\t\tProcessing instance: {instance}")

                          try:
                            img = Image.open(instance_path).convert("RGB")
                            text, bounds = ocr_region(img)

                            if language_folder == 'Chinese':
                              text = ocr_chinese(instance_path)

                            masked_image = mask_image(img, bounds).convert("RGB")
                            caption = generate_caption(masked_image)
                            explaination = generate_meaning(text, caption)

                            result_record = {}
                            result_record["Meme name"] = meme_folder
                            result_record["Language"] = language_folder
                            result_record["Instance"] = instance
                            result_record["Text"] = text
                            result_record["Caption"] = caption
                            result_record["Explaination"] = explaination

                            results.append(result_record)

                          except Exception as e:
                            print(f"\t\tAn error occurred while processing instance {instance} of meme {meme_folder}.")
                            continue


Processing meme: Tuxedo Winnie The Pooh      1/45
	Processing language: English
		Processing instance: 9hml8k.jpg
		Processing instance: 9gldwp.jpg
		Processing instance: 9gfwdv.jpg
		Processing instance: 9h8bl7.jpg
		Processing instance: 9h80me.jpg
		Processing instance: 9go3nh.jpg
		Processing instance: 9gpsto.jpg
	Processing language: French
		Processing instance: Screenshot 2025-01-24 115236.png
		Processing instance: Screenshot 2025-01-24 115304.png
		Processing instance: Screenshot 2025-01-24 115324.png
		Processing instance: Screenshot 2025-01-24 115346.png
		Processing instance: Screenshot 2025-01-24 115404.png
		Processing instance: Screenshot 2025-01-24 115422.png
		Processing instance: Screenshot 2025-01-24 115446.png
	Processing language: Chinese
		Processing instance: 1631368423430.jpg
		Processing instance: 1643205524135.jpg
		Processing instance: 1688441585860.jpg
		Processing instance: 1657887795228.jpg
		Processing instance: 1631444901290.jpg
		Processing instance: 16

In [ ]:
results_path = "results.json"
with open(results_path, "w", encoding="utf-8") as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4)
print(f"Results saved to {results_path}")

Results saved to results.json
